This is a simple example script for 2D x/y stitching using muvis-align and the multiview-stitcher package

In [ ]:
from src.MVSRegistration import MVSRegistration
from src.util import dir_regex

from multiview_stitcher import spatial_image_utils as si_utils
from multiview_stitcher import registration, vis_utils

Initialise registration parameters

In [ ]:
params = {
	'general':
	{
		'chunk_size': [4096, 4096],
		'output':
		{
			'format': 'ome.zarr',
			'tile_size': [4096, 4096],
			'npyramid_add': 4,
			'pyramid_downsample': 2,
			'thumbnail': 'ome.zarr'
		},
	},
	'operations':
	[
		{
			'operation': 'register',
			'input': 'D:/slides/12193/data2d/*.tiff',
			'source_metadata':
			{
				'scale': {'x': 0.004, 'y': 0.004},
				'position': {'y': 'fn[-3]*24', 'x': 'fn[-2]*24'}
			},
            'use_orthogonal_pairs': True,
			'transform_type': 'rigid',
            'scale': 4,
			'method':
			{
				'name': 'sift',
				'gaussian_sigma': 4,
				'inlier_threshold_factor': 0.05,
				'max_trials': 1000,
				'ransac_iterations': 3
			},
			'output': 'stitched/'
		}
	]
}

Get filenames

In [ ]:
operation_params = params['operations'][0]
filenames = dir_regex(operation_params['input'])

filenames

Initialise muvis-align, initialise sims, and pre-process

In [ ]:
reg = MVSRegistration(params['general'])
reg.init_operation('', filenames, operation_params)
sims = reg.init_sims()
sims, norm_sims, _ = reg.preprocess(sims, operation_params)

for filename, sim in zip(filenames, sims):
    print(filename, si_utils.get_origin_from_sim(sim))

Perform registration (using multiview-stitcher)

In [ ]:
results = reg.register(sims)
registered_sims = results['sims']
results['mappings']

Visualise registered sims

In [ ]:
vis_utils.plot_positions(registered_sims, transform_key=reg.reg_transform_key, use_positional_colors=False, view_labels_size=3)

Perform fusion (using multiview-stitcher)

In [ ]:
fused = reg.fuse(registered_sims)

Output fused result

In [ ]:
reg.save(operation_params['output'] + 'test.ome.zarr', fused)